In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Importation et nettoyage des données

In [4]:
df = pd.read_csv(r'C:\Users\pasqu\Documents\GitHub\deepLearning\deep_learning\data\diamonds.csv')
df = df.drop(df.columns[0], axis=1)
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [5]:
cut_map = {
    'Fair': 0,
    'Good': 1,
    'Very Good': 2,
    'Premium': 3,
    'Ideal': 4
}

color_map = {
    'J': 0, 'I': 1, 'H': 2, 'G': 3, 'F': 4, 'E': 5, 'D': 6
}

clarity_map = {
    'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7
}


df['cut'] = df['cut'].map(cut_map)
df['color'] = df['color'].map(color_map)
df['clarity'] = df['clarity'].map(clarity_map)


df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,4,5,1,61.5,55.0,326,3.95,3.98,2.43
1,0.21,3,5,2,59.8,61.0,326,3.89,3.84,2.31
2,0.23,1,5,4,56.9,65.0,327,4.05,4.07,2.31
3,0.29,3,1,3,62.4,58.0,334,4.20,4.23,2.63
4,0.31,1,0,1,63.3,58.0,335,4.34,4.35,2.75


In [6]:
feature_cols = ['color', 'clarity', 'depth', 'table', 'x', 'y', 'z']
target_cols = ['price', 'carat']
X = df[feature_cols]
y = df[target_cols]

X.head()

,color,clarity,depth,table,x,y,z
0,5,1,61.5,55.0,3.95,3.98,2.43
1,5,2,59.8,61.0,3.89,3.84,2.31
2,5,4,56.9,65.0,4.05,4.07,2.31
3,1,3,62.4,58.0,4.20,4.23,2.63
4,0,1,63.3,58.0,4.34,4.35,2.75


In [8]:
y.head()

,price,carat
0,326,0.23
1,326,0.21
2,327,0.23
3,334,0.29
4,335,0.31


In [7]:
X_np = X.values.astype(np.float32)
y_np = y.values.astype(np.float32)

scaler = StandardScaler()

X = scaler.fit_transform(X_np)
y = scaler.fit_transform(y_np)

Modèle


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)

input_dimension = X_train.shape[1]
output_dimension = y_train.shape[1]


In [24]:
p=0.2
modele = nn.Sequential(
    nn.Linear(input_dimension,16),
    nn.ReLU(),
    nn.Dropout(p),
    nn.Linear(16,8),
    nn.ReLU(),
    nn.Linear(8,output_dimension),
)

criterion = (nn.MSELoss())
optimizer = optim.Adam(modele.parameters(), lr=0.01)
summary(modele, input_size=input_dimension, output_size=output_dimension)

Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            128
├─ReLU: 1-2                              --
├─Dropout: 1-3                           --
├─Linear: 1-4                            136
├─ReLU: 1-5                              --
├─Linear: 1-6                            18
Total params: 282
Trainable params: 282
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            128
├─ReLU: 1-2                              --
├─Dropout: 1-3                           --
├─Linear: 1-4                            136
├─ReLU: 1-5                              --
├─Linear: 1-6                            18
Total params: 282
Trainable params: 282
Non-trainable params: 0

In [25]:
 num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass : Calcul des prédictions
    outputs = modele(X_train_tensor)

    # Calcul de la perte (erreur)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass : Calcul des gradients et mise à jour des poids
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.7624
Epoch [20/100], Loss: 0.2701
Epoch [30/100], Loss: 0.1302
Epoch [40/100], Loss: 0.0978
Epoch [50/100], Loss: 0.0766
Epoch [60/100], Loss: 0.0653
Epoch [70/100], Loss: 0.0576
Epoch [80/100], Loss: 0.0526
Epoch [90/100], Loss: 0.0481
Epoch [100/100], Loss: 0.0443


In [23]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


def get_metrics(model, X_tensor, y_true_scaled, scaler_y, dataset_name):
    """
    Calcule les métriques pour un dataset donné (Train ou Test)
    """
    model.eval()
    with torch.no_grad():
        preds_scaled = model(X_tensor).numpy()

    # Retour aux valeurs réelles
    preds_real = scaler_y.inverse_transform(preds_scaled)
    y_true_real = scaler_y.inverse_transform(y_true_scaled)

    results = []
    targets = ['Price ($)', 'Carat (ct)']

    for i, target_name in enumerate(targets):
        y_true = y_true_real[:, i]
        y_pred = preds_real[:, i]

        results.append({
            'Dataset': dataset_name,
            'Cible': target_name,
            'RMSE': round(np.sqrt(mean_squared_error(y_true, y_pred)), 2),
            'MAE': round(mean_absolute_error(y_true, y_pred), 2),
            'R2 Score': round(r2_score(y_true, y_pred), 4)
        })
    return pd.DataFrame(results)

# --- 5. Comparaison Train vs Test ---

# Calcul des métriques sur le Training Set
metrics_train = get_metrics(modele, X_train_tensor, y_train, scaler, "Training")

# Calcul des métriques sur le Test Set
metrics_test = get_metrics(modele, X_test_tensor, y_test, scaler, "Test")

# Concaténation et affichage
final_report = pd.concat([metrics_train, metrics_test], ignore_index=True)

# On trie pour voir Price (Train vs Test) et Carat (Train vs Test) côte à côte
final_report = final_report.sort_values(by='Cible')

print("--- Rapport de Performance (Train vs Test) ---")
print(final_report.to_string(index=False))

--- Rapport de Performance (Train vs Test) ---
 Dataset      Cible    RMSE    MAE  R2 Score
Training Carat (ct)    0.24   0.13    0.7532
    Test Carat (ct)    0.24   0.13    0.7549
Training  Price ($) 1334.50 906.57    0.8881
    Test  Price ($) 1267.32 901.35    0.8990


p=0.2
modele = nn.Sequential(
    nn.Linear(input_dimension,32),
    nn.ReLU(),
    nn.Dropout(p),
    nn.Linear(32,64),
    nn.ReLU(),
    nn.Linear(64,16),
    nn.ReLU(),
    nn.Linear(16,output_dimension),
)

criterion = (nn.MSELoss())
optimizer = optim.Adam(modele.parameters(), lr=0.01)
summary(modele, input_size=input_dimension, output_size=output_dimension)
100 epoch

--- Rapport de Performance (Train vs Test) ---
 Dataset      Cible   RMSE    MAE  R2 Score
Training Carat (ct)   0.04   0.02    0.9927
    Test Carat (ct)   0.04   0.02    0.9946
Training  Price ($) 743.12 394.60    0.9653
    Test  Price ($) 707.80 386.80    0.9685



p=0.2
modele = nn.Sequential(
    nn.Linear(input_dimension,16),
    nn.ReLU(),
    nn.Dropout(p),
    nn.Linear(16,8),
    nn.ReLU(),
    nn.Linear(8,output_dimension),
)

criterion = (nn.MSELoss())
optimizer = optim.Adam(modele.parameters(), lr=0.01)
summary(modele, input_size=input_dimension, output_size=output_dimension)
epoch 100
--- Rapport de Performance (Train vs Test) ---
 Dataset      Cible   RMSE    MAE  R2 Score
Training Carat (ct)   0.07   0.04    0.9798
    Test Carat (ct)   0.06   0.04    0.9859
Training  Price ($) 826.07 486.83    0.9571
    Test  Price ($) 787.17 480.52    0.9610